In [4]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

def corr (x, y):
    print(x+' and '+y+' are corr by '+str(df[x+'_int'].corr(df[y+'_int'])))

df_1=pd.merge(pd.read_csv("FF_Processed_Data_a.csv"),(pd.read_csv("FF_Processed_Data_b.csv")),on='idnum')
print('pair 1...')
df_2=pd.merge((pd.read_csv("FF_Processed_Data_c.csv")),(pd.read_csv("FF_Processed_Data_d.csv")),on='idnum')
print('pair 2...')
df_3=pd.merge(df_1,df_2,on='idnum')
print('pair 3...')
df_4=pd.merge(df_3,(pd.read_csv("FF_Processed_Data_e.csv")),on='idnum')
print('almost...')

df=df_4=pd.merge(df_4,(pd.read_csv("FF_Processed_Data_addi.csv")),on='idnum')
print('Done!')

cf=df[['p6b39_int', 'k6c28_int', 'k6c17_int', 'k6b1d_int', 'k6c3_int', 'p6b16_int', 'cp6edu_int']]
cf=cf.dropna()
cf.k6b1d_int=(4-cf.k6b1d_int)/3
cf.cp6edu_int=(4-cf.cp6edu_int)/3
cf.k6c3_int=cf.k6c3_int/7
cf.p6b16_int=2-cf.p6b16_int

cf.p6b39_int=2-cf.p6b39_int
cf.k6c17_int=(4-cf.k6c17_int)/3
cf.k6c28_int=(4-cf.k6c28_int)/3

def math_predict(a,b,c,d):
# a=how safe they feel at school, with 1 being most and 4 being least. Must be integer.
# b=frequent headaches, where 1=yes and 0=no
# c=education of primary caregiver, either "less", "HS", "coll", or "grad" (with explanations being shown in summary above)
# d=number of nights per week when they eat dinner together
    school_a=50*(df[df.k6b1d_int==a].p6b39_int.mean()-1)
    dad_a=(4-df[df.k6b1d_int==a].k6c28_int.mean())*(100/3)
    if b==0:
        b=2
    school_b=50*(df[df.p6b16_int==b].p6b39_int.mean()-1)
    dad_b=(4-df[df.p6b16_int==b].k6c28_int.mean())*(100/3)
    if c=='less':
        c=1
    elif c=='HS':
        c=2
    elif c=='coll':
        c=3
    elif c=='grad':
        c=4
    school_c=50*(df[df.cp6edu_int==c].p6b39_int.mean()-1)
    dad_c=(4-df[df.cp6edu_int==c].k6c28_int.mean())*(100/3)
    school_d=50*(df[df.k6c3_int==d].p6b39_int.mean()-1)
    dad_d=(4-df[df.k6c3_int==d].k6c28_int.mean())*(100/3)
    school=(school_a+school_b+school_c+school_d)/4
    dad=(dad_a+dad_b+dad_c+dad_d)/4
    print('They have a {0}% chance of misbehaving at school and a {1}% closeness to their father.'.format(round(school,2), round(dad,2)))
    
model = keras.Sequential([
    layers.Dense(24, activation=tf.nn.relu, input_shape=[len(cf.keys())]),
    layers.Dense(24, activation=tf.nn.relu),
    layers.Dense(3, activation=tf.nn.sigmoid)
  ])

optimizer = tf.keras.optimizers.Adam()

model.compile(loss='mean_squared_error',
            optimizer=optimizer,
            metrics=['mean_absolute_error', 'mean_squared_error'])

model = keras.models.load_model('basic_tf_model.h5')

def tf_predict(a,b,c,d):
# a=feeling of safety at school (0 to 1)
# b=probability of dinner with family (0 to 1) also seen as nights per week divided by 7
# c=headaches frequently 0 no and 1 yes
# d=level of parent's education from less than HS (0) to college/graduate (1)
    pred=np.reshape([a,b,c,d], (1,4))
    ans=(model.predict(pred))[0]
    school=100*(1-ans[0])
    school=round(school,3)
    dad=round(ans[1],3)
    mom=round(ans[2],3)
    print(str(school)+'% chance of misbehaving in school, '+str(dad)+' of 1 closeness to father, and '+str(mom)+' of 1 closeness to mother.')

pair 1...
pair 2...
pair 3...
almost...
Done!
Instructions for updating:
Use tf.cast instead.


**Math Model:**
Average scores are 0.12116564417177911 for school, 0.5475330926594464 for dad.

**TensorFlow Model:**
Average scores are 0.7563469107334431 for school, 0.5643672310389005 for dad, and 0.8249631222624045 for mom.

In [5]:
math_predict(4,0,'less',6)
tf_predict(0,0.8,0.02,0)

They have a 11.14% chance of misbehaving at school and a 53.56% closeness to their father.
12.906% chance of misbehaving in school, 0.23 of 1 closeness to father, and 0.822 of 1 closeness to mother.
